In [ ]:
#!pip install spacy en_core_web_md

In [20]:
import spacy
import pandas as pd
nlp = spacy.load("en_core_web_md")

reviews_df=pd.read_csv("../order_review.csv", usecols=[1,2,4,5], parse_dates=["review_creation_date"])
customers_df=pd.read_csv("../customer.csv")
orders_df=pd.read_csv("../order.csv")
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   review_id               99224 non-null  object        
 1   order_id                99224 non-null  object        
 2   review_comment_message  99224 non-null  object        
 3   review_creation_date    99224 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 3.0+ MB


In [21]:
df = reviews_df.merge(orders_df, on="order_id").merge(customers_df, on="customer_id")
df = df[["customer_id", "review_id", "review_creation_date", "review_comment_message"]]
mask = df['review_comment_message'].str.contains("NONE")
df = df[~mask]
    df.head(1)

,customer_id,review_id,review_creation_date,review_comment_message
3,de6dff97e5f1ba84a3cd9a3bc97df5f6,e64fb393e7b32834bb789ff8bb30750e,2017-04-21,Recebi bem antes do prazo estipulado.


In [18]:
def encode(row):
    return nlp(row).vector
df['embedding'] = df.apply(lambda x: encode(x[2]), axis=1) 
df.drop("review_comment_message", inplace=True, axis=1) 
df.head(1)

,customer_id,review_creation_date,embedding
3,de6dff97e5f1ba84a3cd9a3bc97df5f6,2017-04-21,"[0.14603572, -0.32441586, -0.19999127, -0.0086..."


In [22]:
import hsfs
connection = hsfs.connection(engine="hive")
fs = connection.get_feature_store()

embedding_fg = fs.create_feature_group(name="embedding",
                                    primary_key=["customer_id", "review_id"],
                                    event_time=["review_creation_date"],
                                    version=1,
                                    description="Word embedding of customer review",
                                    online_enabled=True)
embedding_fg.save(df)

ExternalClientError: host cannot be of type NoneType, host is a non-optional argument to connect to hopsworks from an external environment.